In [1]:
pip install earthpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import packages
import pandas as pd
import numpy as np
import keras
from keras import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Flatten, Input, GlobalMaxPooling1D
from keras.callbacks import EarlyStopping
from keras import Model
import rasterio
import earthpy.plot as ep
from keras.utils import to_categorical, plot_model, model_to_dot
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import from_levels_and_colors
from sklearn.model_selection import train_test_split

In [46]:
import rasterio
import numpy as np
import earthpy.plot as ep

# File paths for each band
BAND_PATHS = {
    'NDWI': "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/NDWI/Landsat_2022_01_NDWI.tif",
    'NDVI': "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/NDVI/Landsat_2022_01_NDVI.tif",
    'EVI': "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/Bands/EVI_Landsat_2022_01_EVI.tif.tif",
    'NIR': "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/Bands/Landsat_Landsat_2022_01_NIR.tif.tif",
    'RED': "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/Bands/Landsat_Landsat_2022_01_RED.tif.tif",
    'BLUE': "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/Bands/Landsat_Landsat_2022_01_BLUE.tif.tif",
    'SWIR': "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/Bands/Landsat_Landsat_2022_01_SWIR.tif.tif",
    'Correlation': "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/Bands/Landsat_correlation.tif.tif",
    'Dissimilarity': "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/Bands/Landsat_dissimilarity.tif.tif",
    'Contrast': "D:/Jintu/Jintu_ShareFolder/TestDL_2/Landsat Images/Bands/Landsat_contrast.tif.tif"
}

# Create an empty dictionary to store band data
band_data_dict = {}

# Read each band separately and store in the dictionary
for band_name, band_path in BAND_PATHS.items():
    with rasterio.open(band_path) as band:
        band_data_dict[band_name] = band.read(1)  # Read the band as a numpy array

# Define the list of features you want to use
FEATURES = [ 'NDWI', 'NDVI', 'RED', 'NIR', 'BLUE', 'EVI', 'SWIR', 'Dissimilarity', 'Contrast',  'Correlation','Class' ]

# Read the CSV file containing the "class" and "sample" columns
SAMPLE_PATH='D:/Jintu/Jintu_ShareFolder/TestDL_2/Signature/Shapefile/Merge Data/Test_1/Mosaic_Raster/DBF File/texture.csv'
df = pd.read_csv(SAMPLE_PATH)
print(df)

      OBJECTID  OID_                Name  \
0         6761     0  Untitled Placemark   
1         6762     0  Untitled Placemark   
2         6769     0  Untitled Placemark   
3         6770     0  Untitled Placemark   
4         6771     0  Untitled Placemark   
...        ...   ...                 ...   
6799       484     0           Placemark   
6800       485     0           Placemark   
6801       486     0           Placemark   
6802       487     0           Placemark   
6803       488     0           Placemark   

                                             FolderPath  SymbolID  LabelID  \
0                                     Forest.kmz/Forest         0        0   
1                                     Forest.kmz/Forest         0        0   
2                                     Forest.kmz/Forest         0        0   
3                                     Forest.kmz/Forest         0        0   
4                                     Forest.kmz/Forest         0        0   
...

In [25]:
print(df[FEATURES])

          NDWI      NDVI          RED          NIR         BLUE          EVI  \
0     0.427863  0.720709     0.042992     0.264874     0.096792     0.696091   
1     0.479975  0.755801     0.045208     0.325050     0.097747     0.810478   
2     0.380752  0.594101     0.039719     0.199653     0.094167     0.546434   
3     0.420002  0.722578     0.044322     0.275205     0.096042     0.703208   
4     0.404692  0.707466     0.045651     0.337869     0.096315     0.821374   
...        ...       ...          ...          ...          ...          ...   
6799  0.573771 -0.383004 -9999.000000 -9999.000000 -9999.000000 -9999.000000   
6800  0.579719 -0.382816 -9999.000000 -9999.000000 -9999.000000 -9999.000000   
6801  0.591617 -0.373113 -9999.000000 -9999.000000 -9999.000000 -9999.000000   
6802  0.559382 -0.378609 -9999.000000 -9999.000000 -9999.000000 -9999.000000   
6803  0.585269 -0.352914 -9999.000000 -9999.000000 -9999.000000 -9999.000000   

             SWIR  Dissimilarity       

In [41]:
print(df['Class'])

0       1
1       1
2       1
3       1
4       1
       ..
6799    5
6800    5
6801    5
6802    5
6803    5
Name: Class, Length: 6804, dtype: int64


In [42]:
X=df[FEATURES]
Y=df['Class']

In [48]:
#print(X)

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [51]:
df['X'] = 'train'
df.loc[X_test.index, 'X'] = 'test'

In [52]:
print(df)

      OBJECTID  OID_                Name  \
0         6761     0  Untitled Placemark   
1         6762     0  Untitled Placemark   
2         6769     0  Untitled Placemark   
3         6770     0  Untitled Placemark   
4         6771     0  Untitled Placemark   
...        ...   ...                 ...   
6799       484     0           Placemark   
6800       485     0           Placemark   
6801       486     0           Placemark   
6802       487     0           Placemark   
6803       488     0           Placemark   

                                             FolderPath  SymbolID  LabelID  \
0                                     Forest.kmz/Forest         0        0   
1                                     Forest.kmz/Forest         0        0   
2                                     Forest.kmz/Forest         0        0   
3                                     Forest.kmz/Forest         0        0   
4                                     Forest.kmz/Forest         0        0   
...

In [56]:
# Extract unique class values from the "class" column
unique_classes = df['Class'].unique()
print(unique_classes)

# Create the CLASSES list
CLASSES = list(unique_classes)

# Number of classes
N_CLASSES = len(CLASSES)

[1 5 2 3 4]


In [64]:
# Split into train and test based on column
train = df[df['X'] == 'train']
test = df[df['X'] == 'test']

# Split between features and label
train_features = train[FEATURES]
train_label = train['Class']
test_features = test[FEATURES]
test_label = test['Class']

# Function to reshape array input
def reshape_input(array):
  shape = array.shape
  return array.reshape(shape[0], shape[1], 1)

# Convert samples dataframe (pandas) to numpy array
train_input = reshape_input(train_features.to_numpy())
test_input = reshape_input(test_features.to_numpy())

# Also make label data to categorical
train_output = to_categorical(train_label.to_numpy(), N_CLASSES + 1, int)
test_output = to_categorical(test_label.to_numpy(), N_CLASSES + 1, int)

# Show the data shape
print(f'Train features: {train_input.shape}\nTest features: {test_input.shape}\nTrain label: {train_output.shape}\nTest label: {test_output.shape}')

Train features: (5443, 11, 1)
Test features: (1361, 11, 1)
Train label: (5443, 6)
Test label: (1361, 6)


In [66]:
# Make model for our data
# Input shape
train_shape = train_input.shape
input_shape = (train_shape[1], train_shape[2])

# Model parameter
neuron = 128
drop = 0.2
kernel = 2
pool = 2

In [67]:
# Make a sequential model
model = Sequential([
    Input(input_shape),
    Conv1D(neuron * 2, kernel, activation='relu'),  # Increased number of filters
    Conv1D(neuron * 2, kernel, activation='relu'),  # Increased number of filters
    MaxPooling1D(pool),
    Dropout(drop),
    Conv1D(neuron * 4, kernel, activation='relu'),  # Increased number of filters
    Conv1D(neuron * 4, kernel, activation='relu'),  # Increased number of filters
    MaxPooling1D(pool),
    Dropout(drop),
    GlobalMaxPooling1D(),
    Dense(neuron * 4, activation='relu'),  # Increased number of neurons in this layer
    Dropout(drop),
    Dense(neuron * 2, activation='relu'),
    Dropout(drop),
    Dense(N_CLASSES + 1, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 10, 256)           768       
                                                                 
 conv1d_1 (Conv1D)           (None, 9, 256)            131328    
                                                                 
 max_pooling1d (MaxPooling1  (None, 4, 256)            0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 4, 256)            0         
                                                                 
 conv1d_2 (Conv1D)           (None, 3, 512)            262656    
                                                                 
 conv1d_3 (Conv1D)           (None, 2, 512)            524800    
                                                        

In [69]:
# Train the model

# Compline the model
model.compile(
    optimizer='Adam',
    loss='CategoricalCrossentropy',
    metrics=['accuracy']
)

# Create callback to stop training if loss not decreasing
stop = EarlyStopping(
    monitor='loss',
    patience=5
)

# Fit the model
result = model.fit(
    x=train_input, y=train_output,
    validation_data=(test_input, test_output),
    batch_size=2000,
    callbacks=[stop],
    epochs=25,
)

Epoch 1/25
3/3 [==============================] - 3s 712ms/step - loss: 10.0784 - accuracy: 0.2769 - val_loss: 6.6281 - val_accuracy: 0.2234
Epoch 2/25
3/3 [==============================] - 2s 600ms/step - loss: 9.8930 - accuracy: 0.2546 - val_loss: 1.7570 - val_accuracy: 0.2234
Epoch 3/25
3/3 [==============================] - 2s 643ms/step - loss: 5.7677 - accuracy: 0.2758 - val_loss: 1.7544 - val_accuracy: 0.2256
Epoch 4/25
3/3 [==============================] - 2s 602ms/step - loss: 3.5585 - accuracy: 0.2855 - val_loss: 1.7508 - val_accuracy: 0.2256
Epoch 5/25
3/3 [==============================] - 2s 582ms/step - loss: 2.4218 - accuracy: 0.2703 - val_loss: 1.7450 - val_accuracy: 0.2248
Epoch 6/25
3/3 [==============================] - 2s 580ms/step - loss: 2.1078 - accuracy: 0.2574 - val_loss: 1.7358 - val_accuracy: 0.2248
Epoch 7/25
3/3 [==============================] - 2s 584ms/step - loss: 1.9234 - accuracy: 0.2515 - val_loss: 1.7244 - val_accuracy: 0.2256
Epoch 8/25
3/3 [===